In [1]:
from gensim.models import Word2Vec, KeyedVectors
pathToGoogleNews300 = '..//fromBlogOfShaneLynnWordEmbeddingsWithSpacyAndGensim_GoogleNews300//data//GoogleNews-vectors-negative300.bin'
#pathToGoogleNews300 = 'data\GoogleNews-vectors-negative300.bin'
model = KeyedVectors.load_word2vec_format(pathToGoogleNews300, binary=True)

In [2]:
model.doesnt_match("man woman child kitchen".split())

C:\Users\Eric\anaconda3\lib\site-packages\gensim\models\keyedvectors.py:877: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


'kitchen'

In [3]:
model.most_similar(positive=['king', 'woman'], negative=['man'])

[('queen', 0.7118192911148071),
 ('monarch', 0.6189674139022827),
 ('princess', 0.5902431607246399),
 ('crown_prince', 0.5499460697174072),
 ('prince', 0.5377321243286133),
 ('kings', 0.5236844420433044),
 ('Queen_Consort', 0.5235945582389832),
 ('queens', 0.5181134343147278),
 ('sultan', 0.5098593235015869),
 ('monarchy', 0.5087411999702454)]

In [4]:
model.most_similar(positive=['uncle', 'woman'], negative=['man'])

[('aunt', 0.8022665977478027),
 ('mother', 0.7770731449127197),
 ('niece', 0.7684249877929688),
 ('father', 0.7237852215766907),
 ('grandmother', 0.7220370769500732),
 ('daughter', 0.7185647487640381),
 ('sister', 0.7006257772445679),
 ('husband', 0.6982547640800476),
 ('granddaughter', 0.6858304738998413),
 ('nephew', 0.6710714101791382)]

In [5]:
model.most_similar("man")

[('woman', 0.7664012312889099),
 ('boy', 0.6824870109558105),
 ('teenager', 0.6586930751800537),
 ('teenage_girl', 0.6147903203964233),
 ('girl', 0.5921714305877686),
 ('suspected_purse_snatcher', 0.571636438369751),
 ('robber', 0.5585119128227234),
 ('Robbery_suspect', 0.5584409236907959),
 ('teen_ager', 0.5549196004867554),
 ('men', 0.5489763021469116)]

In [6]:
import pandas as pd

pathToGungorVict = '..//..//DS7004//u1720146_DS7004_courseworkCodeAndData//preparationWorks//fromDS7003_Gungor2018VictorianAuthorAttribution_NGram//Gungor_2018_VictorianAuthorAttribution_data-train.csv'
gungorVictRow = pd.read_csv(pathToGungorVict, encoding = 'ISO-8859-1')
#48: Washington Irving/ 13: Frances Hodgson Burnett/ 22: Jacob Abbott/ 25: James Payn/ 36: Oliver Optic

In [7]:
for i in [13, 22, 25, 36, 48]:
    allLines = gungorVictRow.loc[gungorVictRow['author'] == i]
    lines350 = allLines.iloc[0:350]
    linesLast20 = allLines.iloc[-20:]
    try:
        train = train.append(lines350)
        test = test.append(linesLast20)
    except:
        train = lines350
        test = linesLast20
train = train.sample(frac=1, random_state=42).reset_index(drop = True) #1750 lines suffled
test = test.sample(frac=1, random_state=42).reset_index(drop = True) #100 lines suffledtest = authorsLines.sample(frac=0.3, random_state=42)

In [8]:
print(test.index)

RangeIndex(start=0, stop=100, step=1)


In [9]:
print(train.index)

RangeIndex(start=0, stop=1750, step=1)


In [10]:
test

,text,author
0,in his preceding voyage had be with the native...,48
1,it solely to oblige a young woman of genius of...,25
2,died then it was placed over his body when it ...,36
3,this matter he answered coldly and that is one...,25
4,for me to procure the money before the banks c...,25
...,...,...
95,ten thousand persons could be seated in this w...,36
96,stands the celebrated bronze statue of st pete...,36
97,king smiled because birds voices were to be he...,13
98,of a series of and which afterwards distracted...,48


In [11]:
train

,text,author
0,length from her half bound the world chapter x...,36
1,would yet be heard then once again back to our...,25
2,the street s comer but returned saying he was ...,13
3,than i have done â i feel a very natural for h...,25
4,and ihe of hia in ihe county the of after a lo...,48
...,...,...
1745,till the steamer and her prize reached east wh...,36
1746,that money and seems to me tain t thb flag ob ...,36
1747,to the the leading paper in america her reply ...,25
1748,to â rt ci u tliis y in nature be hj all u ii ...,48


In [12]:
import gensim
all_stopwords = set(gensim.parsing.preprocessing.STOPWORDS)

In [30]:
#be careful: nword and counter must be integers --Chiu
import numpy as np  # Make sure that numpy is imported

def makeFeatureVec(words, model, num_features):
    # Function to average all of the word vectors in a given
    # paragraph
    #
    # Pre-initialize an empty numpy array (for speed)
    featureVec = np.zeros((num_features,),dtype="float32")
    #
    nwords = 0
    # 
    # Index2word is a list that contains the names of the words in 
    # the model's vocabulary. Convert it to a set, for speed 
    index2word_set = set(model.wv.index2word)
    index2word_set2 = all_stopwords
    #
    # Loop over each word in the review and, if it is in the model's
    # vocaublary, add its feature vector to the total
    for word in words:
        if word in index2word_set:
            if word in index2word_set2: 
                nwords = nwords + 1
                featureVec = np.add(featureVec,model[word])
    # 
    # Divide the result by the number of words to get the average
    if nwords == 0:
        nwords = 1
    featureVec = np.divide(featureVec,nwords)
    return featureVec


def getAvgFeatureVecs(reviews, model, num_features):
    # Given a set of reviews (each one a list of words), calculate 
    # the average feature vector for each one and return a 2D numpy array 
    # 
    # Initialize a counter
    counter = 0
    # 
    # Preallocate a 2D numpy array, for speed
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
    # 
    # Loop through the reviews
    for review in reviews:
       #
       # Print a status message every 1000th review
        if counter%50 == 0:
            haha = counter; hihi = len(reviews)
            print(f"Review {haha} of {hihi}") #% (counter, len(reviews))
       # 
       # Call the function (defined above) that makes average feature vectors
        #reviewFeatureVecs[counter] = makeFeatureVec(review, model, num_features)
        reviewFeatureVecs[counter] = makeFeatureVec(review, model, num_features)
       #
       # Increment the counter
        counter = counter + 1
    return reviewFeatureVecs

In [31]:
# Import various modules for string cleaning
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords

def review_to_wordlist( review, remove_stopwords=False ):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words.  Returns a list of words.
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(review).get_text()
    #  
    # 2. Remove non-letters
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    #
    # 3. Convert words to lower case and split them
    words = review_text.lower().split()
    #
    # 4. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    #
    # 5. Return a list of words
    return(words)

In [32]:
# ****************************************************************
# Calculate average feature vectors for training and testing sets,
# using the functions we defined above. Notice that we now use stop word
# removal.
num_features = 300
clean_train_reviews = []
for review in train["text"]:
    #clean_train_reviews.append( review_to_wordlist( review, \
        #remove_stopwords=True ))
    clean_train_reviews.append( review_to_wordlist( review ))

trainDataVecs = getAvgFeatureVecs( clean_train_reviews, model, num_features )

print("Creating average feature vecs for test reviews")
clean_test_reviews = []
for review in test["text"]:
    #clean_test_reviews.append( review_to_wordlist( review, remove_stopwords=True ))
    clean_test_reviews.append( review_to_wordlist( review ))

testDataVecs = getAvgFeatureVecs( clean_test_reviews, model, num_features )

Review 0 of 1750


<ipython-input-30-206be6dc17b3>:15: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  index2word_set = set(model.wv.index2word)


Review 50 of 1750
Review 100 of 1750
Review 150 of 1750
Review 200 of 1750
Review 250 of 1750
Review 300 of 1750
Review 350 of 1750
Review 400 of 1750
Review 450 of 1750
Review 500 of 1750
Review 550 of 1750
Review 600 of 1750
Review 650 of 1750
Review 700 of 1750
Review 750 of 1750
Review 800 of 1750
Review 850 of 1750
Review 900 of 1750
Review 950 of 1750
Review 1000 of 1750
Review 1050 of 1750
Review 1100 of 1750
Review 1150 of 1750
Review 1200 of 1750
Review 1250 of 1750
Review 1300 of 1750
Review 1350 of 1750
Review 1400 of 1750
Review 1450 of 1750
Review 1500 of 1750
Review 1550 of 1750
Review 1600 of 1750
Review 1650 of 1750
Review 1700 of 1750
Creating average feature vecs for test reviews
Review 0 of 100
Review 50 of 100


In [33]:
# Fit a random forest to the training data, using 100 trees
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier( n_estimators = 100 )

print("Fitting a random forest to labeled training data...")
forest = forest.fit( trainDataVecs, train["author"] )

# Test & extract results 
result = forest.predict( testDataVecs )

# Write the test results 
output = pd.DataFrame( data={"true_author":test["author"], "pred_author":result} )
output.to_csv( "Word2Vec_AverageVectors.csv", index=False, quoting=3 )

Fitting a random forest to labeled training data...


In [34]:
print(output)

    true_author  pred_author
0            48           36
1            25           25
2            36           36
3            25           25
4            25           25
..          ...          ...
95           36           22
96           36           36
97           13           48
98           48           48
99           25           25

[100 rows x 2 columns]


In [35]:
confusion_matrix = pd.crosstab(output['true_author'], output['pred_author'], rownames=['Actural'], colnames=['Predicted'])
print(confusion_matrix)

Predicted  13  22  25  36  48
Actural                      
13         12   2   2   3   1
22          0  19   1   0   0
25          4   0  16   0   0
36          0   2   1  16   1
48          0   3   0   5  12


In [36]:
import sklearn
sklearn.metrics.confusion_matrix(output['true_author'], output['pred_author'])

array([[12,  2,  2,  3,  1],
       [ 0, 19,  1,  0,  0],
       [ 4,  0, 16,  0,  0],
       [ 0,  2,  1, 16,  1],
       [ 0,  3,  0,  5, 12]], dtype=int64)

In [37]:
sklearn.metrics.precision_recall_fscore_support(output['true_author'], output['pred_author'], average=None)

(array([0.75      , 0.73076923, 0.8       , 0.66666667, 0.85714286]),
 array([0.6 , 0.95, 0.8 , 0.8 , 0.6 ]),
 array([0.66666667, 0.82608696, 0.8       , 0.72727273, 0.70588235]),
 array([20, 20, 20, 20, 20], dtype=int64))

In [38]:
pd.DataFrame(output.true_author.value_counts())

,true_author
48,20
36,20
25,20
22,20
13,20
